In [124]:
import pandas as pd # pandas is a data manipulation library
import numpy as np #provides numerical arrays and functions to manipulate the arrays efficiently
import random
import matplotlib.pyplot as plt # data visualization library
import operator
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_absolute_error
from sklearn.svm import SVR
from collections import defaultdict
from random import shuffle
le = LabelEncoder()

In [10]:
# lets explore movies.csv
m_cols = ['movie_id','movie title','release date','video release date','IMDb URL'
          ,'unknown','Action','Adventure','Animation','Childrens','Comedy',
          'Crime','Documentary','Drama','Fantasy','Film-Noir','Horror','Musical','Mystery','Romance','Sci-Fi',
        'Thriller','War','Western']
data = pd.read_csv('ml-100k/u.item',delimiter='|',names=m_cols,encoding='latin-1')

In [7]:
data.head()

,movie_id,movie title,release date,video release date,IMDb URL,unknown,Action,Adventure,Animation,Childrens,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [22]:
#explore ratings
r_cols = ['user_id','movie_id','rating','timestamp']
data_r = pd.read_csv('ml-100k/u1.base',delimiter='\t',names=r_cols,encoding='latin-1')

movies_r = data.merge(data_r,on = 'movie_id',how = 'inner')

In [23]:
data_r.head()

,user_id,movie_id,rating,timestamp
0,1,1,5,874965758
1,1,2,3,876893171
2,1,3,4,878542960
3,1,4,3,876893119
4,1,5,3,889751712


In [24]:
movies_r.head(5)

,movie_id,movie title,release date,video release date,IMDb URL,unknown,Action,Adventure,Animation,Childrens,...,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,user_id,rating,timestamp
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,1,5,874965758
1,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,2,4,888550871
2,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,6,4,883599478
3,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,13,3,882140487
4,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,16,5,877717833


# Q2, a

In [25]:
ratings = movies_r[['movie_id','movie title','rating']].groupby(['movie_id','movie title']).mean()
ratings = ratings.sort_values(['rating'],ascending=False)
ratings.head(3)

,,rating
movie_id,movie title,
1189,Prefontaine (1997),5.0
1653,Entertaining Angels: The Dorothy Day Story (1996),5.0
1599,Someone Else's America (1995),5.0


# Q2, b

In [40]:
count_rating = movies_r[['movie_id','movie title','rating']].groupby(['movie_id','movie title']).count()
count_rating = count_rating.sort_values(['rating'],ascending=False)
count_rating.head()

,,rating
movie_id,movie title,
50,Star Wars (1977),484
181,Return of the Jedi (1983),422
258,Contact (1997),402


# Q2, c

In [71]:
count_rating = movies_r[['movie_id','movie title','rating']].groupby(['movie_id','movie title']).count()
count_rating = count_rating.reset_index()

count_rating_big_than_4 = movies_r[['movie_id','movie title','rating']][movies_r.rating > 4].groupby(['movie_id','movie title']).count()
count_rating_big_than_4 = count_rating_big_than_4.reset_index()

movies  = pd.DataFrame(columns = ['movie_id','movie title','rating_percent'])

for index, row in count_rating_big_than_4.iterrows():
    total_rating = count_rating.loc[count_rating['movie_id'] == row.movie_id].rating.values[0]
    rating_percent = (float(row.rating) / total_rating) * 100
    movies.loc[index] = [row['movie_id'], row['movie title'],  rating_percent]

movies = movies.sort_values(['rating_percent'],ascending=False)
movies.head(3)

,movie_id,movie title,rating_percent
1034,1293,Star Kid (1997),100.0
1113,1653,Entertaining Angels: The Dorothy Day Story (1996),100.0
986,1189,Prefontaine (1997),100.0


# Q2, d

In [72]:
count_rating = count_rating.sort_values(['rating'],ascending=False)
max_count_rating = count_rating.head(1).rating

new_rating_movies  = pd.DataFrame(columns = ['movie_id','movie title','new rating'])
for index, row in count_rating.iterrows():
    add_rating =  float((row.rating / max_count_rating))
    new_rating =  ratings.loc[ratings['movie_id'] == row.movie_id].rating.values[0] + add_rating - 1
    new_rating_movies.loc[index] = [row['movie_id'], row['movie title'], new_rating]

new_rating_movies = new_rating_movies.sort_values(['new rating'],ascending=False)
new_rating_movies.head(3)
    

,movie_id,movie title,new rating
49,50,Star Wars (1977),4.359504
1284,1293,Star Kid (1997),4.006198
1181,1189,Prefontaine (1997),4.004132


# Q2, e

# Womens:


In [85]:
#merge movies ratings data with users
movie_ratings_data_and_users = movies_ratings_data.merge(data_u,on = 'user_id',how = 'inner')
woman_ratings = movie_ratings_data_and_users[['movie_id','movie title','rating']][movie_ratings_data_and_users.gender == 'F']
woman_ratings.head()

,movie_id,movie title,rating
135,1,Toy Story (1995),4
136,10,Richard III (1995),2
137,14,"Postino, Il (1994)",4
138,25,"Birdcage, The (1996)",4
139,100,Fargo (1996),5


In [119]:
#Q2, e (a)

mean_ratings_women = woman_ratings[['movie_id','movie title','rating']].groupby(['movie_id','movie title']).mean()
mean_ratings_women = mean_ratings_women.sort_values(['rating'],ascending=False)
mean_ratings_women.head(3)

,,rating
movie_id,movie title,
1,Toy Story (1995),3.800000
2,GoldenEye (1995),3.200000
3,Four Rooms (1995),2.538462


In [88]:
#Q2, e (b)

count_rating_womens = woman_ratings[['movie_id','movie title','rating']].groupby(['movie_id','movie title']).count()
count_rating_womens = count_rating_womens.sort_values(['rating'],ascending=False)
count_rating_womens.head(3)


,,rating
movie_id,movie title,
50,Star Wars (1977),125
286,"English Patient, The (1996)",125
288,Scream (1996),118


In [90]:
#Q2, e (c)

count_rating_womens = woman_ratings[['movie_id','movie title','rating']].groupby(['movie_id','movie title']).count()
count_rating_womens = count_rating_womens.reset_index()

count_rating_big_than_4 = woman_ratings[['movie_id','movie title','rating']][woman_ratings.rating > 4].groupby(['movie_id','movie title']).count()
count_rating_big_than_4 = count_rating_big_than_4.reset_index()

movies  = pd.DataFrame(columns = ['movie_id','movie title','rating_percent'])

for index, row in count_rating_big_than_4.iterrows():
    total_rating = count_rating.loc[count_rating['movie_id'] == row.movie_id].rating.values[0]
    rating_percent = (float(row.rating) / total_rating) * 100
    movies.loc[index] = [row['movie_id'], row['movie title'],  rating_percent]

movies = movies.sort_values(['rating_percent'],ascending=False)
movies.head(3)

,movie_id,movie title,rating_percent
846,1599,Someone Else's America (1995),100.0
839,1472,"Visitors, The (Visiteurs, Les) (1993)",50.0
845,1594,Everest (1998),50.0


In [101]:
#Q2, e (d)

count_rating_womens = count_rating_womens.sort_values(['rating'],ascending=False)
max_count_rating = count_rating_womens.head(1).rating

mean_ratings_women = mean_ratings_women.reset_index()
# count_rating = count_rating.reset_index()


new_rating_movies  = pd.DataFrame(columns = ['movie_id','movie title','new rating'])

for index, row in count_rating_womens.iterrows():
    add_rating =  float((row.rating / max_count_rating))
    new_rating =  mean_ratings_women.loc[mean_ratings_women['movie_id'] == row.movie_id].rating.values[0] + add_rating - 1
    new_rating_movies.loc[index] = [row['movie_id'], row['movie title'], new_rating]

new_rating_movies = new_rating_movies.sort_values(['new rating'],ascending=False)
new_rating_movies.head(3)

,movie_id,movie title,new rating
49,50,Star Wars (1977),4.312000
314,318,Schindler's List (1993),4.141161
1121,1150,Last Dance (1996),4.008000


# Q2, e

# Mens:

In [108]:
#merge movies ratings data with users
movie_ratings_data_and_users = movies_ratings_data.merge(data_u,on = 'user_id',how = 'inner')
men_ratings = movie_ratings_data_and_users[['movie_id','movie title','rating']][movie_ratings_data_and_users.gender == 'M']
men_ratings.head()

,movie_id,movie title,rating
0,1,Toy Story (1995),5
1,2,GoldenEye (1995),3
2,3,Four Rooms (1995),4
3,4,Get Shorty (1995),3
4,5,Copycat (1995),3


In [121]:
#Q2, e (a)

mean_ratings_men = men_ratings[['movie_id','movie title','rating']].groupby(['movie_id','movie title']).mean()
mean_ratings_men = mean_ratings_men.sort_values(['rating'],ascending=False)
mean_ratings_men.head(3)

,,rating
movie_id,movie title,
1500,Santa with Muscles (1996),5.0
1189,Prefontaine (1997),5.0
1656,Little City (1998),5.0


In [115]:
#Q2, e (b)

count_rating_mens = men_ratings[['movie_id','movie title','rating']].groupby(['movie_id','movie title']).count()
count_rating_mens = count_rating_mens.sort_values(['rating'],ascending=False)
count_rating_mens.head(3)

,,rating
movie_id,movie title,
50,Star Wars (1977),359
181,Return of the Jedi (1983),318
100,Fargo (1996),300


In [117]:
#Q2, e (c)

count_rating_mens = men_ratings[['movie_id','movie title','rating']].groupby(['movie_id','movie title']).count()
count_rating_mens = count_rating_mens.reset_index()

count_rating_big_than_4 = men_ratings[['movie_id','movie title','rating']][men_ratings.rating > 4].groupby(['movie_id','movie title']).count()
count_rating_big_than_4 = count_rating_big_than_4.reset_index()

movies  = pd.DataFrame(columns = ['movie_id','movie title','rating_percent'])

for index, row in count_rating_big_than_4.iterrows():
    total_rating = count_rating.loc[count_rating['movie_id'] == row.movie_id].rating.values[0]
    rating_percent = (float(row.rating) / total_rating) * 100
    movies.loc[index] = [row['movie_id'], row['movie title'],  rating_percent]

movies = movies.sort_values(['rating_percent'],ascending=False)
movies.head(3)

,movie_id,movie title,rating_percent
921,1293,Star Kid (1997),100.0
962,1500,Santa with Muscles (1996),100.0
982,1653,Entertaining Angels: The Dorothy Day Story (1996),100.0


In [118]:
#Q2, e (d)

count_rating_mens = count_rating_mens.sort_values(['rating'],ascending=False)
max_count_rating = count_rating_mens.head(1).rating

mean_ratings_men = mean_ratings_men.reset_index()

new_rating_movies  = pd.DataFrame(columns = ['movie_id','movie title','new rating'])

for index, row in count_rating_mens.iterrows():
    add_rating =  float((row.rating / max_count_rating))
    new_rating =  mean_ratings_men.loc[mean_ratings_men['movie_id'] == row.movie_id].rating.values[0] + add_rating - 1
    new_rating_movies.loc[index] = [row['movie_id'], row['movie title'], new_rating]

new_rating_movies = new_rating_movies.sort_values(['new rating'],ascending=False)
new_rating_movies.head(3)

,movie_id,movie title,new rating
49,50,Star Wars (1977),4.376045
126,127,"Godfather, The (1972)",4.030356
173,174,Raiders of the Lost Ark (1981),4.016542


# Q3, a

In [92]:
# create test

cols = ['user_id','movie_id','rating','timestamp']
test = pd.read_csv('ml-100k/u1.test',delimiter='\t',names=cols,encoding='latin-1')
test = test[['movie_id','rating']].groupby('movie_id').mean()
test = test.reset_index()
test = data.merge(test,on = 'movie_id',how = 'inner')
test = test.drop(['movie title', 'video release date', 'IMDb URL', 'release date'], axis=1)

test.head(3)

,movie_id,unknown,Action,Adventure,Animation,Childrens,Comedy,Crime,Documentary,Drama,...,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,rating
0,1,0,0,0,1,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,3.797101
1,2,0,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,3.307692
2,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,3.200000


In [84]:
# using Q2, a

ratings = movies_r[['movie_id','movie title','rating']].groupby(['movie_id','movie title']).mean()
ratings = ratings.sort_values(['rating'],ascending=False)

ratings = ratings.reset_index()
ratings = data.merge(ratings, on = ['movie_id','movie title'],how = 'inner')
ratings = ratings.drop(['movie title', 'video release date', 'IMDb URL', 'release date'], axis=1)

ratings.head(3)

,movie_id,unknown,Action,Adventure,Animation,Childrens,Comedy,Crime,Documentary,Drama,...,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,rating
0,1,0,0,0,1,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,3.892950
1,2,0,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,3.180952
2,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,3.000000


In [87]:
# the train using svr
svr = SVR()
predictor = ['unknown','Action','Adventure','Animation','Childrens','Comedy',
          'Crime','Documentary','Drama','Fantasy','Film-Noir','Horror','Musical','Mystery','Romance','Sci-Fi',
        'Thriller','War','Western']
outcome = ['rating']

predictions = svr.fit(ratings[predictor],ratings[outcome]).predict(test[predictor])
print ("MAE: {}".format(mean_absolute_error(test[outcome], predictions)))

C:\Users\qvkg67\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


MAE: 0.6873003008289377


In [93]:
# create test for our rating

cols = ['user_id','movie_id','rating','timestamp']
test = pd.read_csv('ml-100k/u1.test',delimiter='\t',names=cols,encoding='latin-1')


count_rating = movies_r[['movie_id','movie title','rating']].groupby(['movie_id','movie title']).count()
count_rating = count_rating.reset_index()
count_rating = count_rating.sort_values(['rating'],ascending=False)
max_count_rating = count_rating.head(1).rating

new_rating_movies  = pd.DataFrame(columns = ['movie_id','movie title','rating'])
for index, row in count_rating.iterrows():
    add_rating =  float((row.rating / max_count_rating))
    new_rating =  ratings.loc[ratings['movie_id'] == row.movie_id].rating.values[0] + add_rating - 1
    new_rating_movies.loc[index] = [row['movie_id'], row['movie title'], new_rating]

new_rating_movies = new_rating_movies.sort_values(['rating'],ascending=False)


new_rating_movies = new_rating_movies.reset_index()
new_rating_movies = data.merge(new_rating_movies, on = ['movie_id','movie title'],how = 'inner')
new_rating_movies = new_rating_movies.drop(['movie title', 'video release date', 'IMDb URL', 'release date'], axis=1)

test = new_rating_movies
test.head(3)


,movie_id,unknown,Action,Adventure,Animation,Childrens,Comedy,Crime,Documentary,Drama,...,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,index,rating
0,1,0,0,0,1,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,3.684273
1,2,0,1,1,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,1,2.397895
2,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,2,2.154959


In [95]:
# using Q2, d
count_rating = movies_r[['movie_id','movie title','rating']].groupby(['movie_id','movie title']).count()
count_rating = count_rating.reset_index()
count_rating = count_rating.sort_values(['rating'],ascending=False)
max_count_rating = count_rating.head(1).rating

new_rating_movies  = pd.DataFrame(columns = ['movie_id','movie title','rating'])
for index, row in count_rating.iterrows():
    add_rating =  float((row.rating / max_count_rating))
    new_rating =  ratings.loc[ratings['movie_id'] == row.movie_id].rating.values[0] + add_rating - 1
    new_rating_movies.loc[index] = [row['movie_id'], row['movie title'], new_rating]

new_rating_movies = new_rating_movies.sort_values(['rating'],ascending=False)


new_rating_movies = new_rating_movies.reset_index()
new_rating_movies = data.merge(new_rating_movies, on = ['movie_id','movie title'],how = 'inner')
new_rating_movies = new_rating_movies.drop(['movie title', 'video release date', 'IMDb URL', 'release date'], axis=1)
new_rating_movies.head(3)

,movie_id,unknown,Action,Adventure,Animation,Childrens,Comedy,Crime,Documentary,Drama,...,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,index,rating
0,1,0,0,0,1,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,3.684273
1,2,0,1,1,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,1,2.397895
2,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,2,2.154959


In [96]:
# the train using svr
svr = SVR()
predictor = ['unknown','Action','Adventure','Animation','Childrens','Comedy',
          'Crime','Documentary','Drama','Fantasy','Film-Noir','Horror','Musical','Mystery','Romance','Sci-Fi',
        'Thriller','War','Western']
outcome = ['rating']

predictions = svr.fit(new_rating_movies[predictor],new_rating_movies[outcome]).predict(test[predictor])
print ("MAE: {}".format(mean_absolute_error(test[outcome], predictions)))

C:\Users\qvkg67\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


MAE: 0.5956766082578648


# we can see that the our rating get lower MAE score! (see word document)

# Q3, b

In [98]:
all_users = {}
all_users = defaultdict(list)
for index,row in data_r.iterrows():
    all_users[row[0]].append(row[1])

In [110]:
# a

ratings = movies_r[['movie_id','movie title','rating']].groupby(['movie_id','movie title']).mean()
ratings = ratings.reset_index()
ratings = ratings.sort_values(['rating'],ascending=False)

moviesA = ratings['movie_id'].values.tolist()

In [112]:
# b

count_rating = movies_r[['movie_id','movie title','rating']].groupby(['movie_id','movie title']).count()
count_rating = count_rating.reset_index()
count_rating = count_rating.sort_values(['rating'],ascending=False)
moviesB = count_rating['movie_id'].values.tolist()

In [115]:
# c

count_rating = movies_r[['movie_id','movie title','rating']].groupby(['movie_id','movie title']).count()
count_rating = count_rating.reset_index()

count_rating_big_than_4 = movies_r[['movie_id','movie title','rating']][movies_r.rating > 4].groupby(['movie_id','movie title']).count()
count_rating_big_than_4 = count_rating_big_than_4.reset_index()

movies  = pd.DataFrame(columns = ['movie_id','movie title','rating_percent'])

for index, row in count_rating_big_than_4.iterrows():
    total_rating = count_rating.loc[count_rating['movie_id'] == row.movie_id].rating.values[0]
    rating_percent = (float(row.rating) / total_rating) * 100
    movies.loc[index] = [row['movie_id'], row['movie title'],  rating_percent]

movies = movies.sort_values(['rating_percent'],ascending=False)

moviesC = movies['movie_id'].values.tolist()

In [120]:
# d

count_rating = movies_r[['movie_id','movie title','rating']].groupby(['movie_id','movie title']).count()
count_rating = count_rating.reset_index()
count_rating = count_rating.sort_values(['rating'],ascending=False)
max_count_rating = count_rating.head(1).rating

new_rating_movies  = pd.DataFrame(columns = ['movie_id','movie title','rating'])
for index, row in count_rating.iterrows():
    add_rating =  float((row.rating / max_count_rating))
    new_rating =  ratings.loc[ratings['movie_id'] == row.movie_id].rating.values[0] + add_rating - 1
    new_rating_movies.loc[index] = [row['movie_id'], row['movie title'], new_rating]

new_rating_movies = new_rating_movies.sort_values(['rating'],ascending=False)
new_rating_movies.head(3)

moviesD = new_rating_movies['movie_id'].values.tolist()

In [125]:
moviesRand =  movies_r['movie_id'].values.tolist()
shuffle(moviesRand)

In [126]:
allRecomendations = {}
for user_id, movie_list in all_user.items():
    moviesA = [item for item in moviesA if item not in movie_list][0:20]
    moviesB = [item for item in moviesB if item not in movie_list][0:20]
    moviesC = [item for item in moviesC if item not in movie_list][0:20]
    moviesD = [item for item in moviesD if item not in movie_list][0:20]
    moviesRand = [item for item in moviesRand if item not in movie_list][0:20]
    allRecomendations[user_id] = [moviesA,moviesB,moviesC,moviesD,moviesRand]

In [127]:
# example for user 1

allRecomendations[1][0]

[1189,
 1653,
 1599,
 1500,
 1467,
 1201,
 1122,
 1293,
 1449,
 1367,
 408,
 850,
 1594,
 1642,
 318,
 483,
 64,
 12,
 603,
 114]